In [ ]:
import xarray as xr
import hvplot.xarray
import holoviews as hv
import numpy as np
import pandas as pd

from snobedo.lib.dask_utils import start_cluster, client_ip_and_port
from snobedo.snotel import SnotelLocations, CsvParser

from common import SNOBAL_DIR, DATA_DIR, SNOTEL_DIR, COARSEN_OPTS, \
    use_hvplot, HV_PLOT_OPTS, BOKEH_FONT, LEGEND_OPTS, LINE_STYLE

In [ ]:
use_hvplot()

In [ ]:
client = start_cluster(8, 24)
client_ip_and_port(client)

## SNOTEL 

In [ ]:
snotel_sites = SnotelLocations()
snotel_sites.load_from_json(SNOTEL_DIR / 'site-locations/snotel_sites.json')

In [ ]:
schofield_snotel_csv = pd.concat([
    CsvParser.file(
        SNOTEL_DIR / 'wy2021' / f'Schofield/usda-csv/2021-Schofield-Pass.csv',
    ),
    # CsvParser.file(
    #     SNOTEL_DIR / 'wy2022' / f'Schofield/usda-csv/2022-Schofield-Pass.csv',
    # )
])

butte_snotel_csv = pd.concat([
    CsvParser.file(
        SNOTEL_DIR / 'wy2021'  / f'Butte/usda-csv/2021-Butte.csv',
    ),
    # CsvParser.file(
    #     SNOTEL_DIR / 'wy2022'  / f'Butte/usda-csv/2022-Butte.csv',
    # ),
])

taylor_snotel_csv = pd.concat([
    CsvParser.file(
        SNOTEL_DIR / 'wy2021' / f'Taylor/usda-csv/2021-Upper-Taylor.csv',
    ),
    # CsvParser.file(
    #     SNOTEL_DIR / 'wy2022' / f'Taylor/usda-csv/2022-Upper-Taylor.csv',
    # )
])

In [ ]:
irwin_csv = pd.concat([
    CsvParser.file(
        SNOTEL_DIR / 'wy2021' / f'Irwin/usda-csv/2021-Irwin.csv',
    ),
    #     CsvParser.file(
    #     SNOTEL_DIR / 'wy2022' / f'Irwin/usda-csv/2022-Irwin.csv',
    # )
])

irwin_csv['Depth(m)'] = irwin_csv['Depth(m)'] / 10

## Model

In [ ]:
water_year = 'wy2021'

In [ ]:
solar_HRRR = xr.open_mfdataset(
    f'{SNOBAL_DIR}/{water_year}/erw_hrrr_solar/run*/snow.nc',
    parallel=True, chunks={'time': -1},
    preprocess=lambda ds: ds['thickness']
)

butte_snobal_hrrr = solar_HRRR.sel(x=snotel_sites.Butte.lon, y=snotel_sites.Butte.lat, method='nearest').compute()
schofield_snobal_hrrr = solar_HRRR.sel(x=snotel_sites.Schofield.lon, y=snotel_sites.Schofield.lat, method='nearest').compute()
taylor_snobal_hrrr = solar_HRRR.sel(x=snotel_sites.Taylor.lon, y=snotel_sites.Taylor.lat, method='nearest').compute()
irwin_snobal_hrrr = solar_HRRR.sel(x=snotel_sites.Irwin.lon, y=snotel_sites.Irwin.lat, method='nearest').compute()

del solar_HRRR

In [ ]:
solar_HRRR = xr.open_mfdataset(
    f'{SNOBAL_DIR}/{water_year}/erw_hrrr_solar_modis/run*/snow.nc',
    parallel=True, chunks={'time': -1},
    preprocess=lambda ds: ds['thickness']
)

butte_snobal_hrrr_modis = solar_HRRR.sel(x=snotel_sites.Butte.lon, y=snotel_sites.Butte.lat, method='nearest').compute()
schofield_snobal_hrrr_modis = solar_HRRR.sel(x=snotel_sites.Schofield.lon, y=snotel_sites.Schofield.lat, method='nearest').compute()
taylor_snobal_hrrr_modis = solar_HRRR.sel(x=snotel_sites.Taylor.lon, y=snotel_sites.Taylor.lat, method='nearest').compute()
irwin_snobal_hrrr_modis = solar_HRRR.sel(x=snotel_sites.Irwin.lon, y=snotel_sites.Irwin.lat, method='nearest').compute()

del solar_HRRR

In [ ]:
solar_SMRF = xr.open_mfdataset(
    f'{SNOBAL_DIR}/{water_year}/erw/run*/snow.nc',
    preprocess=lambda ds: ds['thickness'],
    parallel=True, chunks={'time': -1},
)

butte_snobal = solar_SMRF.sel(x=snotel_sites.Butte.lon, y=snotel_sites.Butte.lat, method='nearest').compute()
schofield_snobal = solar_SMRF.sel(x=snotel_sites.Schofield.lon, y=snotel_sites.Schofield.lat, method='nearest').compute()
taylor_snobal = solar_SMRF.sel(x=snotel_sites.Taylor.lon, y=snotel_sites.Taylor.lat, method='nearest').compute()
irwin_snobal = solar_SMRF.sel(x=snotel_sites.Irwin.lon, y=snotel_sites.Irwin.lat, method='nearest').compute()

del solar_SMRF

# SNOTEL 

In [ ]:
hv.output(fig='auto', dpi=300)

HV_PLOT_OPTS['ylabel'] = 'Snow Depth (m)'
HV_PLOT_OPTS['xlabel'] = 'Water Year 2021'

x_lim = (np.datetime64('2020-10-20'), np.datetime64('2021-06-25'))
y_lim = (-0.25, 2.5)

## Butte 

## Schofield Pass 

In [ ]:
butte_plot = butte_snobal_hrrr_modis.thickness.squeeze(['x', 'y']).hvplot(label='HRRR Shortwave + MODIS Albedo').opts(
    title='Butte', **LINE_STYLE, **BOKEH_FONT, **HV_PLOT_OPTS
) * \
butte_snobal_hrrr.thickness.squeeze(['x', 'y']).hvplot(label='HRRR Shortwave', **LINE_STYLE) * \
butte_snobal.thickness.squeeze(['x', 'y']).hvplot(label='Time-Decay Albedo', **LINE_STYLE) * \
butte_snotel_csv['Depth(m)'].plot(label='Station Measurements', **LINE_STYLE)

schofield_plot = schofield_snobal_hrrr_modis.thickness.squeeze(['x', 'y']).hvplot(label='HRRR Shortwave + MODIS Albedo').opts(
    title='Schofield Pass', **LINE_STYLE, **BOKEH_FONT, **HV_PLOT_OPTS
) * \
schofield_snobal_hrrr.thickness.squeeze(['x', 'y']).hvplot(label='HRRR Shortwave', **LINE_STYLE) * \
schofield_snobal.thickness.squeeze(['x', 'y']).hvplot(label='Time-Decay Albedo', **LINE_STYLE) * \
schofield_snotel_csv['Depth(m)'].plot(label='Station Measurements', **LINE_STYLE)

(
    butte_plot.opts(xlim=x_lim, ylim=y_lim, **LEGEND_OPTS) + schofield_plot.opts(xlim=x_lim, ylim=y_lim, **LEGEND_OPTS)
).cols(1)

## Upper Taylor 

In [ ]:
taylor_snobal_hrrr_modis.thickness.squeeze(['x', 'y']).hvplot(label='HRRR + MODIS').opts(
    title='Upper Taylor', **LINE_STYLE, **BOKEH_FONT, **HV_PLOT_OPTS
) * \
taylor_snobal_hrrr.thickness.squeeze(['x', 'y']).hvplot(label='HRRR') * \
taylor_snobal.thickness.squeeze(['x', 'y']).hvplot(label='SMRF') * \
taylor_snotel_csv['Depth(m)'].plot(label='Station')

## Irwin 

***NOTE***: Irwin guide is not a SNOTEL station and does not have a QA/QC end of day value. Using a 'rolling daily' window to plot

In [ ]:
irwin_snobal_hrrr_modis.thickness.squeeze(['x', 'y']).hvplot(label='HRRR + MODIS').opts(
    title='Irwin Guide', **LINE_STYLE, **BOKEH_FONT, **HV_PLOT_OPTS
) * \
irwin_snobal_hrrr.thickness.squeeze(['x', 'y']).hvplot(label='HRRR') * \
irwin_snobal.thickness.squeeze(['x', 'y']).hvplot(label='SMRF') * \
irwin_csv['Depth(m)'].rolling('1D').mean().hvplot(label='Station')

In [ ]:
client.shutdown()